In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
sample_2021 = pd.read_csv('/content/drive/MyDrive/Data Sample/sample_2021.csv')
sample_2223 = pd.read_csv('/content/drive/MyDrive/Data Sample/sample_2223.csv')

In [ ]:
from datasets import Dataset

# Assuming sample_2021 is a pandas DataFrame with a column 'text'
dataset_2021 = Dataset.from_pandas(sample_2021)
dataset_2223 = Dataset.from_pandas(sample_2223)

**Roberta Hate Model**

In [ ]:
from transformers import pipeline

# Load the hate speech detection pipeline with the RoBERTa model
model_path = 'cardiffnlp/twitter-roberta-base-hate-latest'
hate_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device=0)

In [ ]:
from datasets import Dataset

# Assuming sample_2021 is a pandas DataFrame with a column 'text'
dataset_2021 = Dataset.from_pandas(sample_2021)
dataset_2223 = Dataset.from_pandas(sample_2223)

In [ ]:
# Define a function to classify hate speech
def classify_hate(batch):
    outputs = hate_classifier(batch['text'], truncation=True)
    batch['label'] = [output['label'] for output in outputs]
    batch['score'] = [output['score'] for output in outputs]
    return batch

In [ ]:
# Process the dataset in batches
batch_size = 150
dataset_2021 = dataset_2021.map(classify_hate, batched=True, batch_size=batch_size)

# Extract the results
hatelabel_2021 = dataset_2021['label']
hatescore_2021 = dataset_2021['score']


In [ ]:
# Process the dataset in batches
batch_size = 150
dataset_2223 = dataset_2223.map(classify_hate, batched=True, batch_size=batch_size)

# Extract the results
hatelabel_2223 = dataset_2223['label']
hatescore_2223 = dataset_2223['score']

**Roberta Toxicity**

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

model_path = "s-nlp/roberta_toxicity_classifier"
toxicity_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device = 0)

In [ ]:
def classify_toxic(batch):
    outputs = toxicity_classifier(batch['text'], truncation=True)
    batch['label'] = [output['label'] for output in outputs]
    batch['score'] = [output['score'] for output in outputs]
    return batch

In [ ]:
# Process the dataset in batches
batch_size = 150
dataset_2021 = dataset_2021.map(classify_toxic, batched=True, batch_size=batch_size)

# Extract the results
toxiclabel_2021 = dataset_2021['label']
toxicscore_2021 = dataset_2021['score']

In [ ]:
# Process the dataset in batches
batch_size = 150
dataset_2223 = dataset_2223.map(classify_toxic, batched=True, batch_size=batch_size)

# Extract the results
toxiclabel_2223 = dataset_2223['label']
toxicscore_2223 = dataset_2223['score']

**Saving files into Drive**

2021 data

In [ ]:
sample_2021['hate_label'] = hatelabel_2021
sample_2021['toxic_label'] = toxiclabel_2021

In [ ]:
# Assuming sample_2021 is your DataFrame
file_path = '/content/drive/My Drive/hateandtoxic/samplehatetoxic_2021.csv'
sample_2021.to_csv(file_path, index=False)  # Save the DataFrame to CSV

2223 data

In [ ]:
sample_2223['hate_label'] = hatelabel_2223
sample_2223['toxic_label'] = toxiclabel_2223

In [ ]:
# Assuming sample_2021 is your DataFrame
file_path = '/content/drive/My Drive/hateandtoxic/samplehatetoxic_2223.csv'
sample_2223.to_csv(file_path, index=False)  # Save the DataFrame to CSV